# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [17]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [18]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aldinga beach,-35.2793,138.4604,16.35,91,99,8.63,AU,1691095953
1,1,castillos,-34.1667,-53.8333,9.85,59,100,2.67,UY,1691095954
2,2,puerto natales,-51.7236,-72.4875,3.60,95,100,10.16,CL,1691095903
3,3,puerto lopez,4.0845,-72.9560,30.67,63,9,1.86,CO,1691095955
4,4,vallenar,-28.5708,-70.7581,22.75,43,6,1.99,CL,1691095955


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo=True, 
    tiles="OSM", 
    s="Humidity", 
    color="City", 
    alpha=0.5,
    frame_width=700,
    frame_height=500)

map_plot_1

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] < 30) & (city_data_df["Max Temp"] > 20) & 
                                (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] < 50)]
#ideal_cities = city_data_df.loc[city_data_df["Max Temp"] < 30]
#ideal_cities

# Drop any rows with null values
ideal_cities_filtered = ideal_cities.dropna(how="any")

# Display sample data
ideal_cities_filtered

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,vallenar,-28.5708,-70.7581,22.75,43,6,1.99,CL,1691095955
33,33,tocopilla,-22.0920,-70.1979,20.20,70,21,3.64,CL,1691095974
39,39,whitehorse,60.7161,-135.0538,22.43,46,40,1.54,CA,1691095981
40,40,fatezh,52.0891,35.8632,24.43,77,25,2.68,RU,1691095981
52,52,guarapari,-20.6667,-40.4975,23.27,89,0,3.60,BR,1691095989
...,...,...,...,...,...,...,...,...,...,...
564,564,bejaia,36.7559,5.0843,27.60,74,0,2.57,DZ,1691096390
566,566,waingapu,-9.6567,120.2641,23.26,92,2,4.20,ID,1691096392
576,576,beruniy,41.6911,60.7525,24.98,65,0,2.06,UZ,1691096424
583,583,kabalo,-6.0500,26.9167,26.68,37,18,1.95,CD,1691096453


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_filtered.copy()
hotel_df = hotel_df.drop(["City_ID", "Max Temp", "Cloudiness", "Wind Speed", "Date"], axis="columns")
#hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Country,Hotel Name
4,vallenar,-28.5708,-70.7581,43,CL,
33,tocopilla,-22.0920,-70.1979,70,CL,
39,whitehorse,60.7161,-135.0538,46,CA,
40,fatezh,52.0891,35.8632,77,RU,
52,guarapari,-20.6667,-40.4975,89,BR,
...,...,...,...,...,...,...
564,bejaia,36.7559,5.0843,74,DZ,
566,waingapu,-9.6567,120.2641,92,ID,
576,beruniy,41.6911,60.7525,65,UZ,
583,kabalo,-6.0500,26.9167,37,CD,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]

    # print(latitude)
    # print(longitude)
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

    #print(json.dumps(name_address, indent=4, sort_keys=True))
    #print(geoapify_key)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vallenar - nearest hotel: Hotel del Marqués
tocopilla - nearest hotel: hotel Isidora
whitehorse - nearest hotel: Town & Mountain Hotel
fatezh - nearest hotel: Фатеж
guarapari - nearest hotel: Osmar Hotel
quirinopolis - nearest hotel: Hotel dos Viajantes
sinop - nearest hotel: No hotel found
the pas - nearest hotel: Wescana Inn
arraial do cabo - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
argostolion - nearest hotel: Ainos Hotel
jales - nearest hotel: No hotel found
moires - nearest hotel: Hotel Olympic
hithadhoo - nearest hotel: Pebbles Inn
catalhoyuk - nearest hotel: No hotel found
astrakhan - nearest hotel: Omega
barabai - nearest hotel: Hotel Bouraq
bayanaul - nearest hotel: Жан
happy valley-goose bay - nearest hotel: Hotel North Two
lewistown - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
kailua-kona - nearest hotel: Kona Seaside Hotel
itaguacu - nearest hotel: Hotel Beira Rio
darwin - nearest hotel: 

,City,Lat,Lng,Humidity,Country,Hotel Name
4,vallenar,-28.5708,-70.7581,43,CL,Hotel del Marqués
33,tocopilla,-22.0920,-70.1979,70,CL,hotel Isidora
39,whitehorse,60.7161,-135.0538,46,CA,Town & Mountain Hotel
40,fatezh,52.0891,35.8632,77,RU,Фатеж
52,guarapari,-20.6667,-40.4975,89,BR,Osmar Hotel
...,...,...,...,...,...,...
564,bejaia,36.7559,5.0843,74,DZ,Hôtel du Nord
566,waingapu,-9.6567,120.2641,92,ID,HOTEL SANDLEWOOD
576,beruniy,41.6911,60.7525,65,UZ,Miymanxana
583,kabalo,-6.0500,26.9167,37,CD,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    s="Humidity",
    color="City",
    alpha=0.5,
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)